In [35]:
import pinecone
import pandas as pd
import time
from datetime import datetime, timedelta
import openai
import json


def get_response_frpm_pinecone_qry(qry, index, model="text-embedding-ada-002"):
    #get embeddings frpm the query
    qry = qry.replace("\n", " ")
    res = openai.Embedding.create(input = [qry], model=model)
    emb = res['data'][0]['embedding']
    # Query the Pinecone database
    res = index.query([emb], top_k=5, include_metadata=True)
    return res

    
# Function query GPT-4
def get_responde_from_gpt4(sys, usr, assist, tmp=0):
    response = openai.ChatCompletion.create(
    model="gpt-4", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
    #model="gpt-3.5-turbo",
    temperature = tmp,
    messages=[
        {"role": "system", "content": sys},
        {"role": "user", "content": usr},
        {"role": "assistant", "content": assist}
    ])
    res = (response['choices'][0]['message']['content'])
    return res

def get_answer_from_query(qry, json_context):
    sys = "You are an helpful assitant that can answer queries about my purchages including sugestions."
    usr = "To answer the question use the following data: " + json_context 
    assist = "query: " + qry
    res = get_responde_from_gpt4(sys, usr, assist, tmp= 0.5)
    return res    

""" Function to determine which days the query might be referred to. This is
required to be able to extract the required data from the database.
"""
def dates_to_be_included_to_answer_the_query(qry, cur_date, cach):
    sys = "You are an helpful assitant that will deduce the dates that this query is refering to."
    usr = "If the last purchase date is " + cur_date
    if len(cach) == 0 :
        usr = usr + """     
        Which dates can be included in the following query. 
        Answer me in the following format: 
        {
        "start_date": "?",
        "end_date": "?"
        } 
        If no date is specified respond with an empty JSON.
        If the dates can be determine do not include any explanation just the data in the specified JSON format.
        """
    else:
        usr = usr + cach +  """
         
        Which dates can be included in the following query. 
        Answer me in the following format: 
        {
        "start_date": "?",
        "end_date": "?"
        } 
        If no date is specified respond with end_date equal to last purchase
        date and start_date equals 6 month before.
        If the dates can be determine do not include any explanation just the data in the specified JSON format.
        """
    assist = "query: " + qry
    res = get_responde_from_gpt4(sys, usr, assist)
    return res


emb_dat = pd.read_csv("data/sales_data_emb.csv")
pinecone.init(api_key="", environment="")
index = pinecone.Index("purchase-index")
openai.api_key = ""
while True:
# Prompt the user for a question
    question = input("What is your question? ")
    print(question)

    #If the user enters 'exit', break out of the loop
    if question == 'exit':
        break
    
    ### IN ORDER TO SIMPLIFY THE PROJECT THIS SECTION WILL NOT BE INCLUDED
    # Process the question and generate an answer
    # Is this a valid question?
    # res = is_a_valid_qry(question)
    # yes = "Yes".upper()
    # if 3 > len(res) or res[:3].upper() != yes: # if is not yes then the question is not compatible with the app
    #     print(res)
    #     time.sleep(3)
    #     continue

    answer = dates_to_be_included_to_answer_the_query(question, "5/27/2016", [])
    all_products_json = {}
    try:  # In case it was possible to determine specific dates
        data = json.loads(answer)
        start_date = data['start_date']
        end_date = data['end_date']

        start_datetime = datetime.strptime(start_date, '%m/%d/%Y').date()
        end_datetime = datetime.strptime(end_date, '%m/%d/%Y').date()
        
        # Extract all documents in the range of dates.
        for index, row in emb_dat.iterrows():
            row_date = datetime.strptime(row['dates'], '%m/%d/%Y').date()
            if start_datetime <= row_date <= end_datetime:
                dict_prod = json.loads(row['products'])
                all_products_json.update(dict_prod)

    except: # In case dates cannot be determined use Pinecone to get the data
        ret_docs = get_response_frpm_pinecone_qry(question, index)
        # Extract all the documents selected by the query
        for match in ret_docs['matches']:
            start_datetime = datetime.strptime(match['id'], '%m/%d/%Y').date()
            for index, row in emb_dat.iterrows():
                row_date = datetime.strptime(row['dates'], '%m/%d/%Y').date()
                if start_datetime == row_date:
                    dict_prod = json.loads(row['products'])
                    all_products_json.update(dict_prod)
                    
    #current_date_products = emb_dat.iloc[-1]['products']

            
    # Display the answer
    json_products_concatenated = json.dumps(all_products_json)
    answer = get_answer_from_query(question, json_products_concatenated)
    print(answer)
    time.sleep(3) 

Tell me all the dates when I bought office related products
Here are the dates when you bought office-related products:

1. January 26, 2016:
   - Developer joke mug - that's a hardware problem (White)
   - Office cube periscope (Black)
   - Packing knife with metal insert blade (Yellow) 9mm
   - Shipping carton (Brown) 500x310x310mm

2. April 30, 2015:
   - Developer joke mug - Oct 31 = Dec 25 (White)
   - Developer joke mug - that's a hardware problem (White)

3. July 7, 2015:
   - Air cushion film 200mmx200mm 325m
   - DBA joke mug - you might be a DBA if (White)

4. June 16, 2015:
   - 32 mm Double sided bubble wrap 10m
   - DBA joke mug - you might be a DBA if (White)
   - Developer joke mug - that's a hardware problem (White)
   - Developer joke mug - this code was generated by a tool (Black)

5. February 6, 2016:
   - 20 mm Double sided bubble wrap 20m
   - DBA joke mug - it depends (White)
   - DBA joke mug - SELECT caffeine FROM mug (White)
   - Developer joke mug - a foo walk